### Notebook for concept detection in neural network

In [24]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

import concepts
import env
from policy import ActorCriticNet

model_name = "net"
session_name = "starship"
board_name = "5x5"
board_size = 5

agents_to_sample = [0, 150, 300, 450, 600, 750, 900, 1050]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = concepts.concept_area_advantage

CONCEPT_NAME = concepts.concept_area_advantage.__name__


In [25]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [27]:
def play_match(agents: list[ActorCriticNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    state = go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.2

    game_over = False

    while not game_over:
        if moves == total_moves:
            break
        # Remove array index 3 and 5 from the current state making it an shape of (4, 5, 5)
        state_copy = np.delete(state, [3, 5], axis=0)
        
        if np.random.random() < sample_ratio:
            if concept_function(state):
                positive_cases.append(state_copy)
            else:
                negative_cases.append(state_copy)
        
        action = agents[current_player].best_action(state)

        state, _, game_over, _ = go_env.step(action)
        moves += 1

        current_player = (current_player + 1) % 2
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

CASES_TO_SAMPLE = 2500 # 25000

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)
            positive_cases.extend(pos)
            negative_cases.extend(neg)
            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

Positive cases:   2%|▏         | 48/2500 [00:13<11:10,  3.66it/s]
Positive cases: 2524it [04:58,  9.52it/s]                          

Positive cases: 2525it [05:11,  9.52it/s]

In [29]:
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 4, 5, 5)
Negative cases:  (2500, 4, 5, 5)


In [ ]:
# Find how many positive cases are identical to other positive cases and count the number of toatal cases
identical_cases = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            identical_cases += 1

print(f"Total cases: {len(positive_cases)}")
print(f"Identical cases: {identical_cases}")

In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [30]:
epochs_to_look_at = [0, 150, 300, 450, 600, 750, 900, 1050]

POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ActorCriticNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_resblock_outputs(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = concepts.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)




Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 266.64it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9472 - val_loss: 0.9092
Epoch 2/50
125/125 [==============================] - 0s 988us/step - loss: 0.8708 - val_loss: 0.8406
Epoch 3/50
125/125 [==============================] - 0s 976us/step - loss: 0.8130 - val_loss: 0.7877
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7671 - val_loss: 0.7451
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7298 - val_loss: 0.7108
Epoch 6/50
125/125 [==============================] - 0s 989us/step - loss: 0.6995 - val_loss: 0.6832
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6751 - val_loss: 0.6611
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6560 - val_loss: 0.6440
Epoch 9/50
125/125 [==============================] - 0s 2ms/step - loss: 0.6413 - val_loss: 0.6315
Epoch 10/50
125/125 [==============================] - 0s 2m

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 293.00it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.1621 - val_loss: 1.0096
Epoch 2/50
125/125 [==============================] - 0s 976us/step - loss: 0.9431 - val_loss: 0.8831
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8438 - val_loss: 0.8102
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7833 - val_loss: 0.7623
Epoch 5/50
125/125 [==============================] - 0s 992us/step - loss: 0.7408 - val_loss: 0.7269
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7075 - val_loss: 0.6980
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6801 - val_loss: 0.6739
Epoch 8/50
125/125 [==============================] - 0s 973us/step - loss: 0.6568 - val_loss: 0.6529
Epoch 9/50
125/125 [==============================] - 0s 946us/step - loss: 0.6362 - val_loss: 0.6337
Epoch 10/50
125/125 [==============================] - 0s 

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 264.96it/s]


...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9876 - val_loss: 0.8999
Epoch 2/50
125/125 [==============================] - 0s 981us/step - loss: 0.8406 - val_loss: 0.8050
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7684 - val_loss: 0.7503
Epoch 4/50
125/125 [==============================] - 0s 953us/step - loss: 0.7231 - val_loss: 0.7125
Epoch 5/50
125/125 [==============================] - 0s 975us/step - loss: 0.6892 - val_loss: 0.6820
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6614 - val_loss: 0.6566
Epoch 7/50
125/125 [==============================] - 0s 979us/step - loss: 0.6378 - val_loss: 0.6345
Epoch 8/50
125/125 [==============================] - 0s 964us/step - loss: 0.6169 - val_loss: 0.6148
Epoch 9/50
125/125 [==============================] - 0s 954us/step - loss: 0.5981 - val_loss: 0.5970
Epoch 10/50
125/125 [==============================] -

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 280.98it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9320 - val_loss: 0.8690
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8400 - val_loss: 0.7972
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7828 - val_loss: 0.7465
Epoch 4/50
125/125 [==============================] - 0s 983us/step - loss: 0.7407 - val_loss: 0.7073
Epoch 5/50
125/125 [==============================] - 0s 976us/step - loss: 0.7066 - val_loss: 0.6751
Epoch 6/50
125/125 [==============================] - 0s 990us/step - loss: 0.6777 - val_loss: 0.6466
Epoch 7/50
125/125 [==============================] - 0s 971us/step - loss: 0.6528 - val_loss: 0.6223
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6314 - val_loss: 0.6012
Epoch 9/50
125/125 [==============================] - 0s 977us/step - loss: 0.6122 - val_loss: 0.5830
Epoch 10/50
125/125 [==============================] - 0

Getting resblock outputs: 100%|██████████| 157/157 [00:00<00:00, 288.36it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.8644 - val_loss: 0.7955
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7834 - val_loss: 0.7314
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7343 - val_loss: 0.6882
Epoch 4/50
125/125 [==============================] - 0s 969us/step - loss: 0.6982 - val_loss: 0.6546
Epoch 5/50
125/125 [==============================] - 0s 992us/step - loss: 0.6687 - val_loss: 0.6265
Epoch 6/50
125/125 [==============================] - 0s 981us/step - loss: 0.6440 - val_loss: 0.6031
Epoch 7/50
125/125 [==============================] - 0s 970us/step - loss: 0.6218 - val_loss: 0.5821
Epoch 8/50
125/125 [==============================] - 0s 972us/step - loss: 0.6025 - val_loss: 0.5639
Epoch 9/50
125/125 [==============================] - 0s 979us/step - loss: 0.5854 - val_loss: 0.5483
Epoch 10/50
125/125 [==============================] -